In [1]:
from keras.layers import Embedding
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras import layers
from keras import backend as K
import keras
import tensorflow as tf


from sqlalchemy import create_engine
import pandas as pd

import logger
from logger import NBatchLogger
import enchant

Using TensorFlow backend.


In [ ]:
'''
This notebook contains code for the advanced preprocessing used for generating expand_join_dict_full. 
The data has not been provided for this part since it needs 7 GB of glove840B and the pickled input combined, but
the algorithm is clear from the flow
'''

#trainX,trainY,testX,testY,X_test,num_vocab, features=pickle.load(open('train_test_prep_full','rb'))
trainX,trainY,testX,testY,X_test,num_vocab, features=pickle.load(open('test_prep_full','rb'))


In [ ]:
embeddings_index = {}
#f = open( './twitter-datasets/glove.twitter.27B/glove.twitter.27B.200d.txt')
#f = open( '/media/keshav/DATA/EPFL/Ubuntu_files/glove.840B.300d.txt')
f = open( './twitter-datasets/glove.840B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    try:
        coefs = np.asarray(values[1:], dtype='float64')
        embeddings_index[word] = coefs
    except:
        pass
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
mult_char=dict()

for char_ in 'abcdefghijklmnopqrstuvwxyz':
    mult_char[char_]=char_*3
embedding_matrix = np.zeros((num_vocab,300))
feature_not_present=[]
for i,feature in enumerate(features):
    feature=feature.replace('#','') 
    feature=feature.replace('"',"")
    feature=feature.replace("'","")
    feature=feature.replace("-","")
    feature=feature.replace("_","")
    for key,value in mult_char.items():
        feature = feature.replace(value,key)
    feature=feature.replace('#','') 
    embedding_vector=embeddings_index.get(feature)
    if(embedding_vector is not None):
        embedding_matrix[i] = embedding_vector
    else:
        feature_not_present.append(feature)
del(embeddings_index) # free up memory 

In [ ]:
print(len(feature_not_present))
slang_dict=pickle.load(open( './twitter-datasets/slang_dict.pkl','rb'))
for feature in feature_not_present:
    if(feature in slang_dict.keys()):
        print(feature + 'is slang for ' + slang_dict[feature])

In [ ]:
def viterbi_segment(text):
    probs, lasts = [1.0], [0]
    for i in range(1, len(text) + 1):
        prob_k, k = max((probs[j] * word_prob(text[j:i]), j)
                        for j in range(max(0, i - max_word_length), i))
        probs.append(prob_k)
        lasts.append(k)
    words = []
    i = len(text)
    while 0 < i:
        words.append(text[lasts[i]:i])
        i = lasts[i]
    words.reverse()
    return words, probs[-1]

def word_prob(word): return dictionary[word] / total
def words(text): return re.findall('[a-z]+', text.lower()) 
dictionary = Counter(words(open('./twitter-datasets/words.txt').read()))
max_word_length = max(map(len, dictionary))
total = float(sum(dictionary.values()))

In [ ]:
words_expand_lookup={}
words_expand_list=[]
for feature in feature_not_present:
    if(len(feature) >  2):
        list_words=viterbi_segment(feature)[0]
        words_expand_list.extend(list_words)
        words_expand_lookup[feature]=list_words
    
final_features_not_present=[]
for i,feature in enumerate(words_expand_list):
    embedding_vector=embeddings_index.get(feature)
    if(embedding_vector is not None):
        pass
    else:
        final_features_not_present.append(feature)

In [ ]:
for key,value in words_expand_lookup.items():
    if '.' in value:
        remaining_words=''.join(value[value.index('.')+1:])
        words_expand_lookup[key].extend(remaining_words.split('.'))
    if( '.' in words_expand_lookup[key] ):
        words_expand_lookup[key].remove('.')
    if(' ' in words_expand_lookup[key] ):
         words_expand_lookup[key].remove(' ')       


In [ ]:
for feature in final_features_not_present:
    if(len(feature) >  0):
        print(feature)

In [ ]:
pickle.dump(dict2,open('./twitter-datasets/expand_join_dict_full','wb'))
dict1=pickle.load(open('./twitter-datasets/expand_join_dict_test','rb'))
dict2=pickle.load(open('./twitter-datasets/expand_join_dict','rb'))

In [ ]:
for key,value in dict1.items():
    if(key not in dict2.keys()):
        dict2[key]=value